# Title: Img_Universal_CV_Dataset_Builder
## Description: 이미지 경로와 라벨을 입력받아 OpenCV로 로드하고, Albumentations 증강을 적용한 뒤 PyTorch Tensor로 반환하는 범용 데이터셋 클래스.
## Input: 
 - img_path_list (List[str]): 이미지 파일 경로 리스트
 - label_list (List[int], Optional): 정답 라벨 리스트 (Test 시 None)
 - transforms (albumentations.Compose, Optional): 적용할 이미지 변환 파이프라인
## Output: 
 - (image, label) tuple: 학습 모드 시
 - image: 추론 모드 시 (label_list가 None일 경우)
## Check Point: 
 - **[중요]** OpenCV는 이미지를 BGR로 읽으므로, `cv2.cvtColor`를 통해 RGB로 변환해야 함. (Pretrained Model은 RGB를 기대함)
 - Albumentations 라이브러리 사용을 전제로 함.

In [ ]:
import cv2
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    """
    이미지 경로를 받아 데이터를 로드하고 전처리를 수행하는 데이터셋 클래스
    """
    def __init__(self, img_path_list, label_list=None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)
        
        if image is None:
            raise FileNotFoundError(f"Image not found at {img_path}")

        # [Architect's Note] cv2는 BGR로 읽히므로 RGB로 변환 필수
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 전처리 및 증강 (Albumentations)
        if self.transforms is not None:
            # Albumentations는 딕셔너리 형태로 결과를 반환함 {'image': ...}
            image = self.transforms(image=image)['image']

        # [Block 3] 라벨 유무에 따른 반환 분기 → 테스트, 훈련 구별
        if self.label_list is not None:
            label = self.label_list[index]
            # 라벨은 텐서로 변환 (LongTensor for Classification)
            # 필요에 따라 torch.tensor(label, dtype=torch.long) 등으로 명시 가능
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

## How to Use
1. **데이터 준비**: 이미지 경로 리스트와 라벨 리스트를 준비합니다.
2. **인스턴스 생성**:
    ```python
    # 학습용 (라벨 있음, 증강 적용)
    train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
    
    # 추론용 (라벨 없음, 증강은 리사이즈/정규화만)
    test_dataset = CustomDataset(test_img_paths, None, test_transform)
    ```
3. **DataLoader 연결**:
    ```python
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    ```

## Troubleshooting
- **`KeyError: 'image'`**: `transforms` 인자에 `albumentations`가 아닌 `torchvision.transforms`를 넣으면 발생합니다. 이 코드는 `albumentations` 전용입니다.
- **이미지 로드 실패**: 경로가 한글이거나 잘못되었을 때 `cv2.imread`는 에러 대신 `None`을 반환하여 이후 코드에서 `AttributeError`가 발생합니다. 위 코드의 `FileNotFoundError` 처리가 이를 방지합니다.